In [1]:
import os 

In [2]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project/recsearch'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/ahmed/project/Kidney-Disease-Classification-Deep-learning-project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    """
    Configuration dataclass for model evaluation stage.

    Attributes:
        root_dir (Path): Root directory for storing evaluation artifacts.
        report_file_path (Path): Path to save the main evaluation report YAML/JSON.
        threshold_accuracy (float): Minimum accuracy threshold to consider model acceptable.
        scores_file_dir (Path): Directory to store evaluation scores.
        scores_file (str): Filename for the evaluation scores JSON.
        report_file_dir (Path): Directory to store detailed evaluation reports.
        report_file (str): Filename for the evaluation report JSON.
        mlflow_tracking_uri (str): MLflow tracking server URI for logging metrics and models.
        mlflow_experiment_name (str): Name of the MLflow experiment.
        all_params (Dict): Dictionary of all hyperparameters and config values for reference/logging.
        param_image_size (List[int]): Image size used during training/evaluation (Height, Width, Channels).
        param_batch_size (int): Batch size used during evaluation.
        training_data_path (Path): Path to the validation dataset for model evaluation.
    """
    root_dir: Path
    report_file_path: Path
    report_file_dir: Path
    report_file: str
    training_data_path: Path

    scores_file_dir: Path
    scores_file: str

    mlflow_tracking_uri: str
    mlflow_experiment_name: str

    all_params: dict
    param_image_size: list
    param_batch_size: int
    threshold_accuracy: float 
     


In [6]:
from project.constants import *
from project.utils import create_directories,read_yaml

In [11]:
class ConfigerationManager:
    """
    Manages the loading and parsing of configuration and parameter YAML files.
    Creates required directories and provides configuration objects 
    for different pipeline components.
    """

    def __init__(self, config=CONFIG_YAML_FILE, param=PARAM_YAML_FILE):
        """
        Initialize the Configuration Manager.

        Args:
            config (str): Path to the main configuration YAML file.
            param (str): Path to the parameters YAML file.

        Raises:
            CustomException: If YAML reading or directory creation fails.
        """
        try:
            self.config = read_yaml(config)
            self.param = read_yaml(param)
            create_directories(self.config.artifacts_root)
        except Exception as e:
            raise Exception(f"Error in ConfigurationManager initialization: {e}")





    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        """
        Create and return the ModelEvaluationConfig dataclass.

        This method:
            1. Reads the model evaluation section from the main config.
            2. Ensures all required directories exist.
            3. Converts string paths to Path objects for OS-independent handling.
            4. Returns a fully populated ModelEvaluationConfig object.
        
        Returns:
            ModelEvaluationConfig: Dataclass containing paths, MLflow info, params, 
                                and evaluation-specific settings.
        """
        config = self.config.model_evaluation

        # Ensure directories exist
        create_directories([
            Path(config.root_dir),
            Path(config.scores_file_dir),
            Path(config.report_file_dir)
        ])

        # Build evaluation config
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            report_file_path=Path(config.report_file_path),
            threshold_accuracy=config.threshold_accuracy,
            scores_file_dir=Path(config.scores_file_dir),
            scores_file=config.scores_file,
            report_file_dir=Path(config.report_file_dir),
            report_file=config.report_file,
            mlflow_tracking_uri=config.mlflow_tracking_uri,
            mlflow_experiment_name=config.mlflow_experiment_name,
            all_params=self.param.to_dict(),
            param_image_size=self.param.IMAGE_SIZE,
            param_batch_size=self.param.BATCH_SIZE,
            training_data_path=Path(
                self.config.data_ingestion.unzip_dir
            ) / "kidney-ct-scan-image"
        )

        return model_evaluation_config


In [12]:
import sys
import os

# Add project root to Python path
#sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname(__file__), "..", "..")))


from pathlib import Path
import tensorflow as tf
import mlflow
from project.entity.config import ModelEvaluationConfig
from project.utils import save_json

class Evaluation:
    """
    Handles model evaluation, saving scores, and logging metrics/models to MLflow.

    Args:
        config (ModelEvaluationConfig): Configuration object for evaluation.
    """

    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def _valid_generator(self):
        """Create validation dataset using tf.data with proper preprocessing."""
        img_size = tuple(self.config.param_image_size[:-1])  # (H, W)
        batch_size = self.config.param_batch_size

        val_ds = tf.keras.utils.image_dataset_from_directory(
            self.config.training_data_path,
            validation_split=0.30,
            subset="validation",
            seed=42,
            image_size=img_size,
            batch_size=batch_size,
            shuffle=False,
            label_mode='categorical',
        )

        normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
        val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
        self.valid_generator = val_ds.prefetch(buffer_size=tf.data.AUTOTUNE)

    @staticmethod
    def load_model(model_path: Path):
        """Load the trained model from the given path."""
        return tf.keras.models.load_model(model_path)

    def evalution(self):
        """
        Evaluate the model on the validation dataset.

        Returns:
            results (list): [loss, accuracy]
        """
        self._valid_generator()
        model = self.load_model(Path("artifacts/training/model.h5"))
        results = model.evaluate(self.valid_generator)
        save_json(
            path=Path(self.config.report_file_path),
            data={"loss": results[0], "accuracy": results[1]}
        )
        return results

    def save_score(self):
        """
        Save evaluation scores (loss and accuracy) as JSON.
        """
        results = self.evalution()
        scores_file_path = Path(self.config.scores_file_dir) / self.config.scores_file
        scores = {"loss": results[0], "accuracy": results[1]}
        save_json(path=scores_file_path, data=scores)

    def log_mlflow(self):
        """
        Log evaluation metrics and trained model to MLflow/DAGsHub.
        """
        # Set DAGsHub credentials (securely)
        os.environ["MLFLOW_TRACKING_USERNAME"] = "Ahmed2797"
        os.environ["MLFLOW_TRACKING_PASSWORD"] = "466cd6e40b4463c19cee521d93d34f35fb915367"

        mlflow.set_tracking_uri(self.config.mlflow_tracking_uri)
        mlflow.set_experiment(self.config.mlflow_experiment_name)

        # Evaluate model
        results = self.evalution()  # [loss, accuracy]

        with mlflow.start_run():
            # Log hyperparameters
            mlflow.log_params(self.config.all_params)

            # Log metrics
            mlflow.log_metric("val_loss", results[0])
            mlflow.log_metric("val_accuracy", results[1])

            # Log trained model
            model = self.load_model(Path("artifacts/training/model.h5"))

            # Use export() for SavedModel format
            export_dir = Path("artifacts/training/model_export")
            export_dir.mkdir(parents=True, exist_ok=True)
            model.export(export_dir)  # TensorFlow 2.12+ method

            # Log the exported model folder as an artifact
            mlflow.log_artifacts(str(export_dir), artifact_path="model")


In [13]:
try:
    config = ConfigerationManager()
    model_evaluation_config = config.get_model_evaluation_config()

    evaluation = Evaluation(config=model_evaluation_config)
    evaluation.save_score()
    evaluation.log_mlflow()
except Exception as e:
    raise Exception(f"Error during model evaluation: {e}")

Found 465 files belonging to 2 classes.
Using 139 files for validation.


2025-11-26 18:41:09.233318: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-11-26 18:41:11.646591: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 205520896 exceeds 10% of free system memory.
2025-11-26 18:41:12.035282: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 205520896 exceeds 10% of free system memory.
2025-11-26 18:41:12.498600: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 51380224 exceeds 10% of free system memory.
2025-11-26 18:41:12.568387: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 102760448 exceeds 10% of free system memory.
2025-11-26 18:41:12.696119: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:84] Allocation of 102760448 exceeds 10% of free system memory.


9/9 ━━━━━━━━━━━━━━━━━━━━ 26s 3s/step - accuracy: 0.7122 - loss: 1.2180


/home/ahmed/miniconda3/envs/kidney/lib/python3.12/site-packages/mlflow/tracking/_tracking_service/utils.py:140: FutureWarning: Filesystem tracking backend (e.g., './mlruns') is deprecated. Please switch to a database backend (e.g., 'sqlite:///mlflow.db'). For feedback, see: https://github.com/mlflow/mlflow/issues/18534
  return FileStore(store_uri, store_uri)
2025/11/26 18:41:36 INFO mlflow.tracking.fluent: Experiment with name 'kidney_classification_experiment' does not exist. Creating a new experiment.


Found 465 files belonging to 2 classes.
Using 139 files for validation.
9/9 ━━━━━━━━━━━━━━━━━━━━ 27s 3s/step - accuracy: 0.7122 - loss: 1.2180
Saved artifact at 'artifacts/training/model_export'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  136002490536336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490536720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490537680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490536912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490538064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490537872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490538448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  136002490538256: TensorSpec(shape=(), dtype=tf.resource, name=None)